In [31]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Activation, Dropout
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
import numpy as np
import cv2
import os
import random
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from keras.utils import to_categorical
from imutils import paths
from sklearn.model_selection import train_test_split

In [33]:
dir_path = '/content/drive/MyDrive/archive/seg_train/seg_train'
categories = ['street','mountain','sea']
data = []
N = 600 #600장씩만 사용

for i in categories:

    paths = os.path.join(dir_path,i)
    print('paths',paths)
    count = 0
    for j in os.listdir(paths):
        img_path = os.path.join(paths, j)
        labels = categories.index(i)
        ext = os.path.splitext(img_path)[1].lower()
        if ext in ['.jpg', '.jpeg', '.png']:
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, (64, 64))
                data.append([img, labels])
                count += 1
            if count >= N:
                break  # 600장만 추가

random.shuffle(data)
print(len(data))

paths /content/drive/MyDrive/archive/seg_train/seg_train/street
paths /content/drive/MyDrive/archive/seg_train/seg_train/mountain
paths /content/drive/MyDrive/archive/seg_train/seg_train/sea
1800


정규화

In [34]:
import numpy as np
X = np.array([i[0] for i in data])
y = np.array([i[1] for i in data])
X = X.astype('float32') / 255.0

In [35]:
x = []
y = []

for features,label in data:
    x.append(features)
    y.append(label)

#Converting lists into numpy arrays
x = np.array(x)
y = np.array(y)
x = x/255.0
x = np.array(x).reshape(-1, 64, 64, 3)
print("Shape of train images is:", x.shape)
print("Shape of labels is:", y.shape)
print(y)
print(x.shape[1:])

Shape of train images is: (1800, 64, 64, 3)
Shape of labels is: (1800,)
[2 1 2 ... 0 1 1]
(64, 64, 3)


In [36]:
model = Sequential()
model.add(Conv2D(32,(3,3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Conv2D(128,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(3,activation='softmax'))
model.summary()


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_28 (Conv2D)              │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_28 (MaxPooling2D) │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_29 (Conv2D)              │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_29 (MaxPooling2D) │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_32 (Dropout)            │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_30 (Conv2D)              │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_30 (MaxPooling2D) │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_33 (Dropout)            │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_9 (Flatten)             │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 64)             │       294,976 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_34 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_35 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 392,579 (1.50 MB)

 Trainable params: 392,579 (1.50 MB)

 Non-trainable params: 0 (0.00 B)

In [37]:

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x, y, epochs=100,batch_size=20,validation_split=0.2)

Epoch 1/100
72/72 ━━━━━━━━━━━━━━━━━━━━ 13s 148ms/step - accuracy: 0.3563 - loss: 1.1387 - val_accuracy: 0.6194 - val_loss: 0.9818
Epoch 2/100
72/72 ━━━━━━━━━━━━━━━━━━━━ 20s 147ms/step - accuracy: 0.5298 - loss: 0.9307 - val_accuracy: 0.6750 - val_loss: 0.7434
Epoch 3/100
72/72 ━━━━━━━━━━━━━━━━━━━━ 21s 157ms/step - accuracy: 0.6193 - loss: 0.7788 - val_accuracy: 0.7333 - val_loss: 0.6117
Epoch 4/100
72/72 ━━━━━━━━━━━━━━━━━━━━ 20s 151ms/step - accuracy: 0.6580 - loss: 0.6903 - val_accuracy: 0.7361 - val_loss: 0.5702
Epoch 5/100
72/72 ━━━━━━━━━━━━━━━━━━━━ 10s 134ms/step - accuracy: 0.6945 - loss: 0.6585 - val_accuracy: 0.7639 - val_loss: 0.5328
Epoch 6/100
72/72 ━━━━━━━━━━━━━━━━━━━━ 10s 130ms/step - accuracy: 0.7036 - loss: 0.6155 - val_accuracy: 0.6833 - val_loss: 0.7467
Epoch 7/100
72/72 ━━━━━━━━━━━━━━━━━━━━ 11s 146ms/step - accuracy: 0.7151 - loss: 0.6244 - val_accuracy: 0.7750 - val_loss: 0.5435
Epoch 8/100
72/72 ━━━━━━━━━━━━━━━━━━━━ 19s 132ms/step - accuracy: 0.7283 - loss: 0.5922 - 

In [38]:

import numpy as np


def image(path):
    print('path',path)
    img = cv2.imread(path)

    new_arr = cv2.resize(img, (64, 64))
    new_arr = np.array(new_arr)
    new_arr = new_arr/255.0
    new_arr = new_arr.reshape(-1, 64, 64, 3)

    return new_arr



In [39]:
dir_path = '/content/drive/MyDrive/archive/seg_test/seg_test'
categories = ['street','mountain','sea']
data = []

correct_cnt=0
N = 100 #요소별 장수
for i in categories:
    paths = os.path.join(dir_path,i)
    files = os.listdir(paths)
    files = files[:N]  # 앞에서 N장만 사용
    for j in files:
        img_path = os.path.join(paths,j)
      #  print(img_path)
        labels = categories.index(i)
      #  print(img_path,labels)
        prediction = model.predict(image(img_path))
        print('labels' ,labels)
        print('classification results' ,prediction.argmax())
        if (labels==prediction.argmax()):
            correct_cnt=correct_cnt+1
print('### correnct cnt out of 100==>',correct_cnt)

path /content/drive/MyDrive/archive/seg_test/seg_test/street/20090.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
labels 0
classification results 0
path /content/drive/MyDrive/archive/seg_test/seg_test/street/20158.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
labels 0
classification results 0
path /content/drive/MyDrive/archive/seg_test/seg_test/street/20209.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
labels 0
classification results 0
path /content/drive/MyDrive/archive/seg_test/seg_test/street/20183.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
labels 0
classification results 0
path /content/drive/MyDrive/archive/seg_test/seg_test/street/20184.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
labels 0
classification results 0
path /content/drive/MyDrive/archive/seg_test/seg_test/street/20080.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
labels 0
classification results 0
path /content/drive/MyDrive/archive/seg_test/seg_test/street/20126.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
labels 0
classification results 